In [1]:
import folium
import folium.plugins as plugins
from folium.plugins import HeatMap
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
#import geopandas
import random


covid_dataframe = pd.read_csv('spread_hist_by_state.csv')
covid_dataframe.columns

Index(['date', 'state', 'positive', 'negative', 'pending',
       'hospitalizedCurrently', 'hospitalizedCumulative', 'inIcuCurrently',
       'inIcuCumulative', 'onVentilatorCurrently', 'onVentilatorCumulative',
       'recovered', 'dataQualityGrade', 'lastUpdateEt', 'hash', 'dateChecked',
       'death', 'hospitalized', 'total', 'totalTestResults', 'posNeg', 'fips',
       'deathIncrease', 'hospitalizedIncrease', 'negativeIncrease',
       'positiveIncrease', 'totalTestResultsIncrease'],
      dtype='object')

In [2]:
covid_data = covid_dataframe[['date', 'state', 'positive', 'negative','recovered', 'dataQualityGrade', 'lastUpdateEt']].loc[covid_dataframe['dataQualityGrade'] != 'F']
covid_data.head(3)

,date,state,positive,negative,recovered,dataQualityGrade,lastUpdateEt
0,20200513,AK,383.0,30266.0,338.0,B,5/13/2020 0:00
1,20200513,AL,10617.0,125755.0,NaN,B,5/13/2020 0:00
2,20200513,AR,4164.0,69051.0,3220.0,A,5/12/2020 18:33


In [4]:
state_geo_df = pd.read_csv('state_latlong.csv')
covid_df = pd.merge(covid_data, state_geo_df, how='left', left_on="state", right_on='state')
#covid_df
#print(covid_df.loc[covid_df['latitude'] ==  ][:])


In [5]:
plot_df = covid_df[['latitude','longitude','date','positive', 'negative', 'state', 'recovered']].sort_values(by='date', axis=0, ascending=True).fillna(0)
print(plot_df.loc[plot_df['latitude'] == 0]['state'].unique())
plot_df

[]


,latitude,longitude,date,positive,negative,state,recovered
3913,47.751074,-120.740139,20200122,1.0,0.0,WA,0.0
3912,47.751074,-120.740139,20200123,1.0,0.0,WA,0.0
3911,47.751074,-120.740139,20200124,1.0,0.0,WA,0.0
3910,47.751074,-120.740139,20200125,1.0,0.0,WA,0.0
3909,47.751074,-120.740139,20200126,1.0,0.0,WA,0.0
...,...,...,...,...,...,...,...
32,41.492537,-99.901813,20200513,8692.0,41033.0,NE,0.0
31,47.551493,-101.002012,20200513,1647.0,47298.0,ND,969.0
30,35.759573,-79.019300,20200513,15816.0,194641.0,NC,9115.0
43,18.198719,-66.352675,20200513,2329.0,0.0,PR,0.0


In [6]:
#gdf = geopandas.GeoDataFrame(
#    plot_df, geometry=geopandas.points_from_xy(plot_df.longitude, plot_df.latitude))
#gdf

In [7]:
help(plugins.HeatMapWithTime)
#help(folium.Icon)

Help on class HeatMapWithTime in module folium.plugins.heat_map_withtime:

class HeatMapWithTime(folium.map.Layer)
 |  HeatMapWithTime(data, index=None, name=None, radius=15, min_opacity=0, max_opacity=0.6, scale_radius=False, gradient=None, use_local_extrema=False, auto_play=False, display_index=True, index_steps=1, min_speed=0.1, max_speed=10, speed_step=0.1, position='bottomleft', overlay=True, control=True, show=True)
 |  
 |  Create a HeatMapWithTime layer
 |  
 |  Parameters
 |  ----------
 |  data: list of list of points of the form [lat, lng] or [lat, lng, weight]
 |      The points you want to plot. The outer list corresponds to the various time
 |      steps in sequential order. (weight is in (0, 1] range and defaults to 1 if
 |      not specified for a point)
 |  index: Index giving the label (or timestamp) of the elements of data. Should have
 |      the same length as data, or is replaced by a simple count if not specified.
 |  name : string, default None
 |      The name 

In [8]:
main_map = folium.Map(
    [plot_df['latitude'].mean(),plot_df['longitude'].mean()],
    tiles='stamentoner',
    zoom_start=4,
    max_opacity=0.9,
    blur=4
)



#heat_data_pos_grp = []
#heat_data_neg_grp = []
#heat_data_rec_grp = []
date_indx = []

layer_types = [{'title' : 'Covid : Tested Positive',
                'status' : 'positive',
                'show' : True,
                'plot_data' : [],
                'bgr' : [0.0,1.0,0.0],
                'rad' : 3
               },
               {'title' : 'Covid : Tested Negative',
                'status' : 'negative',
                'show' : False,
                'plot_data' : [],
                'bgr' : [0.5,0.5,0.5],
                'rad' : 2
               },
               {'title' : 'Covid : Recovered',
                'status' : 'recovered',
                'show' : False,
                'plot_data' : [],
                'bgr' : [0.0,0.0,1.0],
                'rad' : 1
               }
              ]

for date_stp in plot_df['date'].unique():
    heat_df = plot_df.loc[plot_df['date'] == date_stp].dropna(axis=0, subset=['latitude', 'longitude', 'positive', 'negative', 'recovered', 'date'])
    dt = str(date_stp)
    y = int(dt[:4])
    m = int(dt[4:6])
    d = int(dt[6:])
    date_indx.append(datetime(y,m,d).strftime("%Y-%m-%d"))    
    heat_data_pos_date = []
    heat_data_neg_date = []
    heat_data_rec_date = []
    for row in heat_df.itertuples():
        heat_data_pos_date.append([row.latitude * random.uniform(1.0004,1.001), row.longitude * random.uniform(1.0004,1.001), row.positive])
        heat_data_neg_date.append([row.latitude * random.uniform(1.0004,1.001), row.longitude * random.uniform(1.0004,1.001), row.negative])
        heat_data_rec_date.append([row.latitude * random.uniform(1.0004,1.001), row.longitude * random.uniform(1.0004,1.001), row.recovered])
    layer_types[0]['plot_data'].append(heat_data_pos_date)
    layer_types[1]['plot_data'].append(heat_data_neg_date)
    layer_types[2]['plot_data'].append(heat_data_rec_date)

print(f'layer_types[0]["plot_data"] [{len(layer_types[0]["plot_data"])}] date_indx [{len(date_indx)}]')
#print(f'layer_types[0][3] {"--"* 40}\n{layer_types[0][3]}')


for layer_type in layer_types:

    #feature_sub_grp = folium.plugins.FeatureGroupSubGroup(map_Feature_Group,layer_type[0]+'_grp', show=layer_type[2])
    map_Feature_Group = folium.FeatureGroup(name=layer_type['title'], show=layer_type['show'])
    main_map.add_child(map_Feature_Group)
    #main_map.add_child(feature_sub_grp)

    Heat_mapw_time = plugins.HeatMapWithTime(
        data = layer_type['plot_data'],
        index = date_indx,
        auto_play = True,
        use_local_extrema = True,
        radius = layer_type['rad'],
        min_opacity = 0.5,
        scale_radius = True,
        name = layer_type['title'],
        gradient={layer_type['bgr'][0]: 'blue', layer_type['bgr'][1]: 'lime', layer_type['bgr'][2]: 'red'}
    )
     #control=layer_type[2]

    main_map.add_child(Heat_mapw_time)

    
#marker_cluster = MarkerCluster().add_to(map_Feature_Group)
#folium.TileLayer(layer_type[0]).add_to(map)    
# add the layer control
folium.LayerControl().add_to(main_map)


main_map

layer_types[0]["plot_data"] [113] date_indx [113]


In [ ]:
# extract lat, lon, and magnitude for the folium heatmap
from folium.plugins import HeatMap
df_copy = df[df.month>4].copy()
df_copy['count'] = 1
base_map = generateBaseMap()
HeatMap(data=df_copy[['pickup_latitude', 'pickup_longitude', 'count']].groupby(['pickup_latitude', 'pickup_longitude']).sum().reset_index().values.tolist(), radius=8, max_zoom=13).add_to(base_map)

In [ ]:
    dt = str(date_stp)
    y = int(dt[:4])
    m = int(dt[4:6])
    d = int(dt[6:])
    date_indx.append(datetime(y,m,d).strftime("%Y-%m-%d"))

In [ ]:
industriesDF = pd.read_csv('merged_industries.csv')
industriesDF

In [ ]:
'''


#g1 = folium.plugins.FeatureGroupSubGroup(mcg, 'g1') # First group, in mcg
#g2 = folium.plugins.FeatureGroupSubGroup(mcg, 'g2') # Second group, in mcg
#m.add_child(mcg)
#m.add_child(g1)
#m.add_child(g2)
#g1.add_child(folium.Marker([0,0]))
#g2.add_child(folium.Marker([0,1]))
#folium.LayerControl().add_to(m)



#  zoom_start=6
params = {
    'location':[plot_df['latitude'].mean(),plot_df['longitude'].mean()],
    'tiles':'stamentoner',
    'min_lat':plot_df['latitude'].min(),
    'max_lat':plot_df['latitude'].max(),
    'min_lon':plot_df['longitude'].min(),
    'max_lon':plot_df['longitude'].max()
}
#        min_lat=plot_df['latitude'].min(),
#        max_lat=plot_df['latitude'].max(),
#        min_lon=plot_df['longitude'].min(),
#        max_lon=plot_df['longitude'].max(),
covid_map = folium.Map(
        [plot_df['latitude'].mean(),plot_df['longitude'].mean()],
        tiles='stamentoner',
        zoom_start=5
    )


#heat_df = plot_df.dropna(axis=0, subset=['latitude','longitude'])
#heat_data = [[row['latitude'],row['longitude']] for index, row in heat_df.iterrows()]

heat_df = plot_df.loc[plot_df['date'] == 20200315].dropna(axis=0, subset=['latitude','longitude', 'total'])
print(f'heat_df {len(heat_df)}')
#heat_df['weight'] = heat_df['total'].astype(float)
#heat_data = [[[row['latitude'],row['longitude']] for index, row in heat_df[heat_df['weight'] == i].iterrows()] for i in range(0,plot_df['total'].max())]

#heat_time = list(heat_df.dropna(axis=0, subset=['date']))
heat_time =  [[row['date']] for index, row in heat_df.iterrows()]
#heat_time =  [[index] for index, row in heat_df.iterrows()]

#heat_time = plot_df[])#.toList()
heat_data = []
for row in heat_df.itertuples():
    #print(row.total)
    heat_data.append([row.latitude,row.longitude, row.total])# for i in range(0, row.total)]
#    [heat_data.append([row.latitude,row.longitude, row.total]) for i in range(0, row.total)]
#heat_data = [[row['latitude'],row['longitude'],row['total']] for index, row in heat_df.iterrows()]
len(heat_data)

                
#     for index, row in heat_df.iterrows()]
print(heat_data[:3])

HeatMap(heat_data).add_to(covid_map)
covid_map'''